In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.feature_selection import feature_selection as fs

In [2]:
df = pd.read_csv('../../data/datos_limpios.csv')

In [3]:
#Viendo tablas de frecuencias para escoger categorías de referencia
sost = pd.crosstab(index=df["sost"],columns="count")
sost

col_0,count
sost,
autonomas,41069
estatal,315183
federal,86592
particulares,117782


In [4]:
ent = pd.crosstab(index=df["nom_ent"],columns="count")
ent

col_0,count
nom_ent,
aguascalientes,7807
baja_california,14592
baja_california_sur,4074
campeche,4763
chiapas,29359
chihuahua,14770
coahuila,14598
colima,3769
distrito_federal,18444


In [5]:
subsist = pd.crosstab(index=df["subsist"],columns="count")
subsist

col_0,count
subsist,
bach_estatal_dge-cge,80929
bachillerato_autonomo,41154
cecyte,45570
cobach,85019
colbach/otros_federales,31651
conalep,34144
dgeti,50999
particulares/a.c.,117782
telebach/emsad,73378


In [6]:
primaria = pd.crosstab(index=df["p67"],columns="count")
primaria

col_0,count
p67,
1.0,289876
2.0,168779
3.0,66435
4.0,12353
5.0,23183


In [7]:
secundaria = pd.crosstab(index=df["p70"],columns="count")
secundaria

col_0,count
p70,
1.0,262857
2.0,163809
3.0,121601
4.0,3813
5.0,3613
6.0,1312
7.0,527
8.0,3094


In [8]:
#Definiendo las variables independientes y dependientes

data = df.reindex(columns=['p1','p2','p6','p7','p8','p10','p11','p12','p13','p14','p15',
                           'p16','p17','p18','p19','p20','p21','p22','p23','p24','p25',
                           'p26','p27','p28','p29','p30','p31','p32','p33','p34','p35',
                           'p36','p37','p38','p39','p40','p41','p42','p43','p44','p45',
                           'p46','p47','p48','p49','p50','p51','p52','p53','p54','p55',
                           'p56','p57','p58','p59','p60','p61','p62','p63','p64','p65',
                           'p66','p68','p69','p71','p72','p73','p75','p76','p77','p78',
                           'p79','p80','p81','p82','p83','p84','p85','p86','p87','p88',
                           'p89','p90','p91','p92','p93','p94','p95','p96','p97','p98',
                           'p99','p100','p101','p102','p103','p104','p105','p106','p107',
                           'p108','p109','p110','p111','p112','p113','p114','p115','p116',
                           'p117','p118','p119','p120','p121','p122','p123','p124','p125',
                           'p126','p127','p128','p129','p130','p131','p132','p133','p134',
                           'p135','p136','p137','p138','p139','p140','p141','p142','p143',
                           'p144','p145','p146','p147','p148','p149','p150','p151','p152',
                           'p153','p154','nom_ent_aguascalientes','nom_ent_baja_california',
                           'nom_ent_baja_california_sur','nom_ent_campeche','nom_ent_chiapas',
                           'nom_ent_chihuahua','nom_ent_coahuila','nom_ent_colima',
                           'nom_ent_durango','nom_ent_estado_de_mexico','nom_ent_guanajuato',
                           'nom_ent_guerrero','nom_ent_hidalgo','nom_ent_jalisco',
                           'nom_ent_michoacan','nom_ent_morelos','nom_ent_nayarit',
                           'nom_ent_nuevo_leon','nom_ent_oaxaca','nom_ent_puebla',
                           'nom_ent_queretaro','nom_ent_quintana_roo','nom_ent_san_luis_potosi',
                           'nom_ent_sinaloa','nom_ent_sonora','nom_ent_tabasco','nom_ent_tamaulipas',
                           'nom_ent_tlaxcala','nom_ent_veracruz','nom_ent_yucatan','nom_ent_zacatecas',
                           'modalidad_bachillerato_tecnologico','modalidad_tecnico_profesional',
                           'sost_autonomas','sost_federal','sost_particulares',
                           'subsist_bach_estatal_dge-cge','subsist_bachillerato_autonomo',
                           'subsist_cecyte','subsist_cobach','subsist_colbach/otros_federales',
                           'subsist_conalep','subsist_dgeti','subsist_telebach/emsad','p3_B',
                           'p3_C','p4_A','p4_C','p5_A','p5_C','p9_B','p9_C','p9_D','p9_E',
                           'p9_F','p9_G','p67_B','p67_C','p67_D','p67_E','p70_B','p70_C',
                           'p70_D','p70_E','p70_F','p70_G','p70_H','p74_B','p74_C','p74_D',
                           'p74_E','p74_F'])
t_hmat = np.asarray(df.reindex(columns=['hmat']))
t_hlec = np.asarray(df.reindex(columns=['hlec']))

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560626 entries, 0 to 560625
Columns: 219 entries, p1 to p74_F
dtypes: float64(147), int64(72)
memory usage: 936.7 MB


In [10]:
#Corriendo un modelo lineal para hmat
model_hmat = sm.OLS(t_hmat, data).fit()

In [11]:
model_hmat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     1113.
Date:                Sat, 23 May 2020   Prob (F-statistic):               0.00
Time:                        18:13:27   Log-Likelihood:            -6.1753e+05
No. Observations:              560626   AIC:                         1.235e+06
Df Residuals:                  560407   BIC:                         1.238e+06
Df Model:                         218                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
p1                                     0.2360      0.002     97.569      0.000       0.231       0.241
p2                                    -0.0321      0.001    -25.910      0.000      -0.035      -0.030
p6                                    -0.0162      0.005     -3.063      0.002      -0.027      -0.006
p7                                     0.0104      0.001      9.705      0.000       0.008       0.012
p8                                     0.0229      0.001     36.241      0.000       0.022       0.024
p10                                    0.0479      0.001     37.198      0.000       0.045       0.050
p11                                    0.0220      0.001     18.586      0.000       0.020       0.024
p12                                    0.0090      0.001      9.307      0.000       0.007       0.011
p13                                   -0.0122      0.001    -11.173      0.000      -0.014      -0.010
p14                                    0.0084      0.001      7.690      0.000       0.006       0.010
p15                                   -0.0033      0.001     -3.050      0.002      -0.005      -0.001
p16                                    0.0329      0.001     37.688      0.000       0.031       0.035
p17                                    0.0059      0.001      5.054      0.000       0.004       0.008
p18                                    0.0187      0.001     16.236      0.000       0.016       0.021
p19                                    0.0006      0.001      0.602      0.547      -0.001       0.003
p20                                    0.0070      0.001      5.999      0.000       0.005       0.009
p21                                   -0.0269      0.001    -24.021      0.000      -0.029      -0.025
p22                                    0.0012      0.001      1.016      0.310      -0.001       0.004
p23                                    0.0071      0.002      4.124      0.000       0.004       0.010
p24                                   -0.0116      0.002     -7.314      0.000      -0.015      -0.008
p25                                   -0.0108      0.001     -7.672      0.000      -0.014      -0.008
p26                                    0.0029      0.001      2.013      0.044    7.76e-05       0.006
p27                                   -0.0209      0.001    -14.594      0.000      -0.024      -0.018
p28                                   -0.0054      0.001     -4.128      0.000      -0.008      -0.003
p29                                    0.0298      0.001     23.643      0.000       0.027       0.032
p30                                   -0.0222      0.001    -19.491      0.000      -0.024      -0.020
p31                                   -0.0401      0.001    -28.341      0.000      -0.043      

In [12]:
#Corriendo el algoritmo de backward_elimination para hmat y hlec
data, significativas_hmat,significativas_hlec = fs.backward_elimination_datos(0.05)

	-> Importando los datos
	-> Seleccionando variables para incluir en la regresión
	-> Haciendo backward_elimination para hmat
	-> Haciendo backward_elimination para hlec


In [13]:
#Obteniendo las variables no significativas de hmat
no_significativas_hmat=list(set(data) - set(significativas_hmat))
no_significativas_hmat

['sost_autonomas',
 'p128',
 'p54',
 'p117',
 'p35',
 'nom_ent_guerrero',
 'p67_D',
 'p70_F',
 'p147',
 'p22',
 'p120',
 'subsist_dgeti',
 'p50',
 'nom_ent_colima',
 'p99',
 'nom_ent_tabasco',
 'p70_H',
 'p4_A',
 'p106',
 'nom_ent_nuevo_leon',
 'p47',
 'p19',
 'p66',
 'p70_G',
 'p5_C',
 'p115',
 'p60']

In [14]:
#Corriendo un modelo lineal para hlec
model_hlec = sm.OLS(t_hlec, data).fit()

In [15]:
model_hlec.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     1551.
Date:                Sat, 23 May 2020   Prob (F-statistic):               0.00
Time:                        18:34:10   Log-Likelihood:            -5.6489e+05
No. Observations:              560626   AIC:                         1.130e+06
Df Residuals:                  560407   BIC:                         1.133e+06
Df Model:                         218                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
p1                                     0.0712      0.002     32.334      0.000       0.067       0.076
p2                                    -0.0196      0.001    -17.321      0.000      -0.022      -0.017
p6                                     0.0140      0.005      2.918      0.004       0.005       0.023
p7                                     0.0011      0.001      1.179      0.238      -0.001       0.003
p8                                     0.0134      0.001     23.223      0.000       0.012       0.014
p10                                    0.0559      0.001     47.703      0.000       0.054       0.058
p11                                    0.0107      0.001      9.985      0.000       0.009       0.013
p12                                   -0.0012      0.001     -1.304      0.192      -0.003       0.001
p13                                   -0.0106      0.001    -10.698      0.000      -0.013      -0.009
p14                                    0.0037      0.001      3.706      0.000       0.002       0.006
p15                                   -0.0014      0.001     -1.366      0.172      -0.003       0.001
p16                                    0.0302      0.001     38.065      0.000       0.029       0.032
p17                                   -0.0065      0.001     -6.154      0.000      -0.009      -0.004
p18                                    0.0203      0.001     19.373      0.000       0.018       0.022
p19                                   -0.0017      0.001     -1.840      0.066      -0.004       0.000
p20                                    0.0030      0.001      2.814      0.005       0.001       0.005
p21                                   -0.0457      0.001    -44.881      0.000      -0.048      -0.044
p22                                    0.0041      0.001      3.715      0.000       0.002       0.006
p23                                    0.0348      0.002     22.327      0.000       0.032       0.038
p24                                   -0.0074      0.001     -5.147      0.000      -0.010      -0.005
p25                                   -0.0101      0.001     -7.881      0.000      -0.013      -0.008
p26                                    0.0315      0.001     23.695      0.000       0.029       0.034
p27                                   -0.0236      0.001    -18.142      0.000      -0.026      -0.021
p28                                   -0.0180      0.001    -15.134      0.000      -0.020      -0.016
p29                                    0.0406      0.001     35.401      0.000       0.038       0.043
p30                                   -0.0355      0.001    -34.202      0.000      -0.038      -0.033
p31                                   -0.0452      0.001    -35.123      0.000      -0.048      

In [16]:
#Obteniendo las variables no significativas de hlec
no_significativas_hlec=list(set(data) - set(significativas_hlec))
no_significativas_hlec

['sost_autonomas',
 'p117',
 'p12',
 'nom_ent_tamaulipas',
 'p89',
 'p104',
 'p75',
 'p146',
 'p50',
 'nom_ent_guanajuato',
 'p70_E',
 'nom_ent_colima',
 'p7',
 'nom_ent_chihuahua',
 'p4_A',
 'nom_ent_nuevo_leon',
 'p15',
 'p74_F',
 'p132',
 'p70_B',
 'p19',
 'subsist_colbach/otros_federales',
 'p70_G']